## Aim:

1. Replace the R script to muliply apsimx files via the Edit feature. 
2. First step is to figure out the critical parameters that need to be changed.  
3. Second step is to change the general parameters,  
    such as Weather, Clock, SowingDates, MaximumHeight, Coverfile path    
4. Third step is to setup the combination of soil parameters. 

There are two ways to do the multiplication:   
1. Edit only the sowing rules based on the current slurp setup.  
2. Provide all possible combinations of skl, klr, rfv and modify a trimmed version slurp to have one apsimx file for one set of parameters. 

A combination of both might be more suitable for Richard's data. 
Because: 
The parameter **Depth** does not match the current slurp setup. 
There will be multiple paths to modify all kls 


In [2]:
#load packages
import sqlite3
import re
import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import math

# Try call edit in python to see if it speeds up than R 


In [8]:

os.system('cp C:/Data/Master/01raw-data/ApsimxFiles/20200517BaseSlurp.apsimx \
            C:/Data/Master/03processed-data/apsimxFiles/test.apsimx ')

0

In [2]:
con = sqlite3.connect('../03processed-data/Richard.sqlite3')
mycur = con.cursor() 
mycur.execute("SELECT name FROM sqlite_master WHERE type='table' ORDER BY name;")
(mycur.fetchall())

[('ES',),
 ('Phenology',),
 ('SoilWater',),
 ('SowingDates',),
 ('biomass',),
 ('met_AshleyDene',),
 ('met_Iversen12',),
 ('root',)]

In [7]:
water = pd.read_sql('Select * from SoilWater',  con)
water = water.drop(['Season','Rep', 'Plot', 'Seed', 'DAS','SWC'], axis = 1)

In [61]:
water['SW(1)']
y = [str(x) for x in water['SW(1)']]
# [a for a in y.str not in '[]' ]
[str(a) for a in y  if a not in "["]

['17.4',
 '21.1',
 '17.9',
 '25.2',
 '18.4',
 '19.9',
 '27.0',
 '16.9',
 '12.5',
 '16.9',
 '15.3',
 '23.0',
 '16.8',
 '15.0',
 '25.0',
 '15.7',
 '10.3',
 '17.6',
 '17.3',
 '23.3',
 '19.9',
 '19.8',
 '23.6',
 '16.6',
 '8.3',
 '14.3',
 '13.9',
 '21.2',
 '17.1',
 '15.1',
 '24.1',
 '14.6',
 '8.7',
 '12.0',
 '11.1',
 '15.8',
 '14.2',
 '12.1',
 '21.5',
 '9.6',
 '18.7',
 '13.0',
 '14.3',
 '7.3',
 '7.3',
 '10.3',
 '9.8',
 '10.1',
 '9.4',
 '10.3',
 '21.7',
 '8.6',
 '12.3',
 '17.5',
 '13.1',
 '7.8',
 '7.3',
 '12.4',
 '10.9',
 '12.1',
 '15.1',
 '13.1',
 '21.9',
 '11.5',
 '16.9',
 '21.8',
 '17.5',
 '12.9',
 '4.9',
 '10.3',
 '8.5',
 '12.6',
 '9.8',
 '11.7',
 '20.0',
 '9.9',
 '16.2',
 '21.2',
 '14.9',
 '12.0',
 '3.9',
 '11.3',
 '9.7',
 '11.9',
 '12.0',
 '9.1',
 '19.4',
 '10.6',
 '17.1',
 '22.2',
 '17.4',
 '13.6',
 '19.3',
 '21.0',
 '20.9',
 '16.2',
 '4.7',
 '8.2',
 '6.6',
 '8.9',
 '8.4',
 '5.9',
 '9.0',
 '7.3',
 '12.8',
 '18.7',
 '15.0',
 '8.2',
 '16.2',
 '18.4',
 '18.6',
 '13.1',
 '3.6',
 '8.1',
 '

In [4]:
water['Clock.Today'] = pd.to_datetime(water['Clock.Today'])

In [5]:
# indexing
water = water.pivot_table(index = 'Clock.Today', 
                          columns=['Experiment', 'SowingDate'])

In [6]:
# Rename the indexs
water.columns.set_names('Depth', level=0, inplace=True)
water.columns

MultiIndex([('SW(1)', 'AshleyDene',  'SD1'),
            ('SW(1)', 'AshleyDene', 'SD10'),
            ('SW(1)', 'AshleyDene',  'SD2'),
            ('SW(1)', 'AshleyDene',  'SD3'),
            ('SW(1)', 'AshleyDene',  'SD4'),
            ('SW(1)', 'AshleyDene',  'SD5'),
            ('SW(1)', 'AshleyDene',  'SD6'),
            ('SW(1)', 'AshleyDene',  'SD7'),
            ('SW(1)', 'AshleyDene',  'SD8'),
            ('SW(1)', 'AshleyDene',  'SD9'),
            ...
            ('SW(9)',  'Iversen12',  'SD1'),
            ('SW(9)',  'Iversen12', 'SD10'),
            ('SW(9)',  'Iversen12',  'SD2'),
            ('SW(9)',  'Iversen12',  'SD3'),
            ('SW(9)',  'Iversen12',  'SD4'),
            ('SW(9)',  'Iversen12',  'SD5'),
            ('SW(9)',  'Iversen12',  'SD6'),
            ('SW(9)',  'Iversen12',  'SD7'),
            ('SW(9)',  'Iversen12',  'SD8'),
            ('SW(9)',  'Iversen12',  'SD9')],
           names=['Depth', 'Experiment', 'SowingDate'], length=460)

In [167]:
AD = water.filter(regex='Ash')

In [202]:
SowingDates.AD = pd.to_datetime(SowingDates.AD)
ADSD = SowingDates.loc[:,['AD']].set_index('AD')
ADSD

""
AD
2010-10-21
2010-11-09
2010-12-08
2011-01-13
2011-02-03
2011-10-10
2011-11-07
2011-12-09
2012-01-10


In [203]:
test = water.stack().reset_index('SowingDate')
test

Depth       SowingDate      SW(1)               SW(10)                SW(11)  \
Experiment             AshleyDene Iversen12 AshleyDene  Iversen12 AshleyDene   
Clock.Today                                                                    
2010-10-04         SD1        NaN   24.4375        NaN  29.737500        NaN   
2010-11-07         SD2        NaN   37.5500        NaN  32.300000        NaN   
2010-11-09         SD1     20.400       NaN  10.125000        NaN   9.675000   
2010-11-09         SD2     20.550       NaN   9.512500        NaN   8.733333   
2010-11-16         SD1     16.925       NaN  10.125000        NaN   9.675000   
...                ...        ...       ...        ...        ...        ...   
2012-07-12         SD1     27.025   31.3500  10.258970  15.798134  10.117417   
2012-07-12         SD2     27.525   33.9500   8.617785  15.477369   8.713053   
2012-07-12         SD3     26.725   34.3750   9.693781  13.819046   9.712001   
2012-07-12         SD4     26.025   33.3250  10.081971  11.643555  10.277973   
2012-07-12         SD5     26.775   33.3250   9.976413  10.703515  10.306053   

Depth                      SW(12)                SW(13)  ...      SW(5)  \
Experiment   Iversen12 AshleyDene  Iversen12 AshleyDene  ... AshleyDene   
Clock.Today                                              ...              
2010-10-04   31.600000        NaN  30.862500        NaN  ...        NaN   
2010-11-07   32.650000        NaN  31.775000        NaN  ...        NaN   
2010-11-09         NaN   9.566667        NaN   9.983333  ...  10.987500   
2010-11-09         NaN   9.037500        NaN   9.512500  ...  11.437500   
2010-11-16         NaN   9.566667        NaN   9.983333  ...  10.987500   
...                ...        ...        ...        ...  ...        ...   
2012-07-12   13.802965  10.515580  11.898018  10.937942  ...  12.507679   
2012-07-12   16.375567   9.002520  14.914502   9.298873  ...  12.113551   
2012-07-12   11.877661  10.011075  10.650952  10.317968  ...  13.467418   
2012-07-12    8.834491  10.656218   9.858525  11.042251  ...  12.718977   
2012-07-12   14.254444  10.016752  12.687165   9.734606  ...  11.974454   

Depth                       SW(6)                 SW(7)                 SW(8)  \
Experiment   Iversen12 AshleyDene  Iversen12 AshleyDene  Iversen12 AshleyDene   
Clock.Today                                                                     
2010-10-04   26.250000        NaN  27.012500        NaN  29.237500        NaN   
2010-11-07   20.600000        NaN  20.325000        NaN  24.125000        NaN   
2010-11-09         NaN  10.237500        NaN   9.837500        NaN   9.987500   
2010-11-09         NaN  10.300000        NaN  10.100000        NaN   9.575000   
2010-11-16         NaN  10.237500        NaN   9.837500        NaN   9.987500   
...                ...        ...        ...        ...        ...        ...   
2012-07-12   22.752681  11.826762  21.270136  11.042039  17.038894  11.132101   
2012-07-12   25.550316  11.312237  24.862137  10.436897  20.724628   9.506494   
2012-07-12   26.399688  11.942856  25.758929  11.547449  22.128188  11.216957   
2012-07-12   22.365564  11.460772  19.641506  10.798474  15.049302  10.517676   
2012-07-12   25.292672  11.072835  22.590500  10.346008  19.169730   9.837641   

Depth                       SW(9)             
Experiment   Iversen12 AshleyDene  Iversen12  
Clock.Today                                   
2010-10-04   30.612500        NaN  31.387500  
2010-11-07   27.687500        NaN  30.150000  
2010-11-09         NaN  10.300000        NaN  
2010-11-09         NaN   9.291667        NaN  
2010-11-16         NaN  10.300000        NaN  
...                ...        ...        ...  
2012-07-12   14.404833  10.820058  14.250229  
2012-07-12   17.493417   9.371624  15.418443  
2012-07-12   19.657516  10.197305  16.176761  
2012-07-12   14.535333  10.319041  13.617989  
2012-07-12   14.841761   9.470730  11.006047  

[456 rows x 47 columns]

In [209]:
pd.merge_asof( ADSD,water,  left_index=True, right_index=True, direction='nearest').filter(regex='Ash').stack().reset_index('AD')

,AD,0
"(SW(1), AshleyDene, SD1)",2010-11-09,20.4000
"(SW(1), AshleyDene, SD2)",2010-11-09,20.5500
"(SW(10), AshleyDene, SD1)",2010-11-09,10.1250
"(SW(10), AshleyDene, SD2)",2010-11-09,9.5125
"(SW(11), AshleyDene, SD1)",2010-11-09,9.6750
...,...,...
"(SW(5), AshleyDene, SD10)",2012-02-17,11.3250
"(SW(6), AshleyDene, SD10)",2012-02-17,9.0250
"(SW(7), AshleyDene, SD10)",2012-02-17,8.2375
"(SW(8), AshleyDene, SD10)",2012-02-17,7.9000


In [151]:
SD.pivot_table(index = 'Clock.Today', columns=['Experiment', 'SowingDate'], )

DataError: No numeric types to aggregate

In [115]:
# Check unique values in the factors 
# print (water.apply(lambda x: x.nunique()))

for i in water.columns[0:8:2]:
    print(water.loc[:, i].unique())

['AshleyDene' 'Iversen12']
['2010/11' '2011/12']
[1. 2. 3. 4.]
['CS']


In [116]:
# indexing 
water = water.set_index(['Experiment', 'Clock.Today', 'SowingDate'
                        ]).drop(['Season','Rep', 'Plot', 'Seed', 'DAS','SWC'
                                ], axis = 1)

## # Initial soil water 
### Bring in the sowing dates data


In [170]:
SowingDates = pd.read_sql('Select * from SowingDates',  con)

In [99]:
SD = SowingDates.melt(id_vars = 'SD')
SD.columns = ['SowingDate', 'Experiment', 'Clock.Today']
SD.loc[SD['Experiment'] == 'AD', 'Experiment'] = 'AshleyDene'
SD.loc[SD['Experiment'] == 'I12', 'Experiment'] = 'Iversen12'
SD['Clock.Today'] = pd.to_datetime(SD['Clock.Today'])
# SD = SD.set_index(['SowingDate', 'Experiment', 'Clock.Today'])
SD

,SowingDate,Experiment,Clock.Today
0,SD1,AshleyDene,2010-10-21
1,SD2,AshleyDene,2010-11-09
2,SD3,AshleyDene,2010-12-08
3,SD4,AshleyDene,2011-01-13
4,SD5,AshleyDene,2011-02-03
5,SD6,AshleyDene,2011-10-10
6,SD7,AshleyDene,2011-11-07
7,SD8,AshleyDene,2011-12-09
8,SD9,AshleyDene,2012-01-10
9,SD10,AshleyDene,2012-02-17


In [117]:
SWmean = water.groupby(['Experiment', 'Clock.Today', 'SowingDate'
                       ]).mean().reset_index()

In [109]:
# SWmean = SWmean.reindex(sorted(SWmean.columns), axis=1)
SD = SD.reindex(sorted(SD.columns), axis=1).set_index(['Clock.Today'])

KeyError: "None of ['Clock.Today'] are in the columns"

In [123]:
SWmean.loc[SWmean['Experiment'],].index.get_loc('2010-10-21', method = 'nearest')

,Experiment,SowingDate,SW(1),SW(2),SW(3),SW(4),SW(5),SW(6),SW(7),SW(8),...,SW(14),SW(15),SW(16),SW(17),SW(18),SW(19),SW(20),SW(21),SW(22),SW(23)
Clock.Today,,,,,,,,,,,,,,,,,,,,,
2010-11-09,AshleyDene,SD1,20.400,20.400,16.800000,13.375000,10.987500,10.237500,9.837500,9.987500,...,10.633333,10.612500,11.437500,11.700000,11.150000,10.937500,10.987500,11.012500,11.362500,11.781250
2010-11-09,AshleyDene,SD2,20.550,20.550,18.875000,14.262500,11.437500,10.300000,10.100000,9.575000,...,10.204167,10.287500,9.766667,9.541667,9.420833,10.212500,10.679167,10.704167,10.949306,11.405035
2010-11-16,AshleyDene,SD1,16.925,16.925,16.800000,13.375000,10.987500,10.237500,9.837500,9.987500,...,10.633333,10.612500,11.437500,11.700000,11.150000,10.937500,10.987500,11.012500,11.362500,11.781250
2010-11-16,AshleyDene,SD2,18.125,18.125,18.875000,14.262500,11.437500,10.300000,10.100000,9.575000,...,10.120833,10.304167,9.929167,9.512500,9.670833,10.141667,10.437500,10.837500,10.741667,11.326042
2010-11-24,AshleyDene,SD1,17.125,17.125,17.025000,13.725000,11.050000,10.400000,9.725000,10.175000,...,10.333333,11.612500,11.887500,12.400000,11.750000,11.487500,11.237500,11.462500,12.262500,14.306250
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2012-07-12,AshleyDene,SD1,27.025,27.025,17.919765,14.525151,12.507679,11.826762,11.042039,11.132101,...,11.048355,11.162215,10.936958,10.715749,10.238141,9.774411,9.613701,9.456728,10.068340,10.648862
2012-07-12,AshleyDene,SD2,27.525,27.525,20.757333,15.200673,12.113551,11.312237,10.436897,9.506494,...,9.179560,9.072182,8.381527,7.724510,7.865482,8.016736,8.333014,8.654471,9.563204,10.816617
2012-07-12,AshleyDene,SD3,26.725,26.725,19.402622,15.157370,13.467418,11.942856,11.547449,11.216957,...,10.262212,10.215995,9.575531,8.955347,9.271781,9.594781,9.999233,10.433735,10.488916,9.899838


In [ ]:
# https://janakiev.com/blog/python-shell-commands/